In [30]:
import cv2
from skimage.io import imread
from skimage.transform import resize
from skimage.feature import hog
from skimage import exposure
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
import os
import numpy as np
from preprocessing import *
from skimage.morphology import skeletonize

In [31]:
def dist(x, y, x1, y1):
    return ((x - x1)**2)**(0.5) + ((y - y1)**2) ** (0.5)
def LaplacianEdge(img_binary,name=""):
    kernel_size = 3
    ddepth = cv2.CV_16S
    img_edge = cv2.Laplacian(img_binary, ddepth, ksize=kernel_size)
    img_edge = 255 - img_edge
    #cv2.imwrite(f"output/{name}_final_otsu.png",img_edge)
    return img_edge

In [32]:
def Train_NN(X,Y):
    print("Start Training ")
    N = len(X[0])
    X = np.array(X)
    Y = np.array(Y)
    x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=4, stratify=Y)
    clf = MLPClassifier(alpha=1e-05, hidden_layer_sizes=(2048,1024),random_state=1,solver='lbfgs',max_iter=10000)
    clf = clf.fit(x_train, y_train)
    print("Finished Fitting")
    y_pred_test = clf.predict(x_test)
    accuracy_test = np.mean(y_test==y_pred_test) * 100
    
    y_pred_train = clf.predict(x_train)
    accuracy_train = np.mean(y_train==y_pred_train) * 100
    return accuracy_train,accuracy_test,clf

In [4]:

base_dir='ACdata_base/'

fonts = os.listdir(base_dir)
X,Y=[],[]
for font in fonts:
    h,w = 0,0
    data = os.listdir(base_dir+font)
    print("curFont",font)
    for img in data:
        img_dir = base_dir+font+'/'+img
        imgGray = cv2.imread(img_dir, cv2.IMREAD_GRAYSCALE)
        resized_img = resize(imgGray, (110, 200))
        fd, hog_image = hog(resized_img, orientations=9, pixels_per_cell=(8, 8),cells_per_block=(2, 2), visualize=True)
        X.append(fd)
        Y.append(int(font))


curFont 8
curFont 5
curFont 6
curFont 7
curFont 9
curFont 3
curFont 2
curFont 1
curFont 4


In [10]:
accuracyTrain,accuracyTest,ModelNN = Train_NN(X,Y)
print(accuracyTrain,accuracyTest)

Start Training 
Finished Fitting


In [38]:
imgGray = cv2.imread("0210.jpg", cv2.IMREAD_GRAYSCALE)
resized_img = (resize(imgGray, (110, 200))*255).astype(np.uint8)
binarizedImg =  Binarize_Histogram(resized_img,img_dir)
edgeImg = LaplacianEdge(binarizedImg,name="")
skeleton = 255 - skeletonize(1-binarizedImg/255)*255
cv2.imwrite("edges.png",edgeImg)
cv2.imwrite("skeleton.png",skeleton)
fd_edges, hog_edges = hog(edgeImg, orientations=9, pixels_per_cell=(8, 8),cells_per_block=(2, 2), visualize=True)
fd_sk, hog_sk = hog(skeleton, orientations=9, pixels_per_cell=(8, 8),cells_per_block=(2, 2), visualize=True)
cv2.imwrite("hog_edges.png",hog_edges)
cv2.imwrite("hog_sk.png",hog_sk)

True

In [28]:
print(fd_edges.shape)
print(fd_sk.shape)

(11880,)
(11880,)


In [39]:
###########################Edges only hog
base_dir='ACdata_base/'
fonts = os.listdir(base_dir)
X,Y=[],[]
for font in fonts:
    h,w = 0,0
    data = os.listdir(base_dir+font)
    print("curFont",font)
    for img in data:
        img_dir = base_dir+font+'/'+img
        imgGray = cv2.imread(img_dir, cv2.IMREAD_GRAYSCALE)
        resized_img = (resize(imgGray, (110, 200))*255).astype(np.uint8)
        binarizedImg =  Binarize_Histogram(resized_img,img_dir)
        edgeImg = LaplacianEdge(binarizedImg,name="")
        fd, hog_image = hog(edgeImg, orientations=9, pixels_per_cell=(8, 8),cells_per_block=(2, 2), visualize=True)
        X.append(fd)
        Y.append(int(font))


curFont 8
curFont 5
curFont 6
curFont 7
curFont 9
curFont 3
curFont 2
curFont 1
curFont 4


In [40]:
accuracyTrain,accuracyTest,ModelNN = Train_NN(X,Y)
print(accuracyTrain,accuracyTest)

Start Training 
Finished Fitting
100.0 86.7298578199052


In [42]:
###########################Skeleton hog
base_dir='ACdata_base/'
fonts = os.listdir(base_dir)
X,Y=[],[]
for font in fonts:
    h,w = 0,0
    data = os.listdir(base_dir+font)
    print("curFont",font)
    for img in data:
        img_dir = base_dir+font+'/'+img
        imgGray = cv2.imread(img_dir, cv2.IMREAD_GRAYSCALE)
        resized_img =  (resize(imgGray, (110, 200))*255).astype(np.uint8)
        binarizedImg =  Binarize_Histogram(resized_img,img_dir)
        skeleton = 255 - skeletonize(1-binarizedImg/255)*255
        fd, hog_image = hog(skeleton, orientations=9, pixels_per_cell=(8, 8),cells_per_block=(2, 2), visualize=True)
        X.append(fd)
        Y.append(int(font))

curFont 8
curFont 5
curFont 6
curFont 7
curFont 9
curFont 3
curFont 2
curFont 1
curFont 4


In [43]:
accuracyTrain,accuracyTest,ModelNN = Train_NN(X,Y)
print(accuracyTrain,accuracyTest)

Start Training 
Finished Fitting
100.0 78.90995260663507
